In [1]:
import bigframes.pandas as bpd
import parse

In [8]:

from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.bucket("garrettwu_bucket")
blob_in = bucket.blob("pdf/dremel.pdf")

# with open("/tmp/data/dremel.pdf", "w") as f:
#     f.write(" ")
bts = blob_in.download_to_filename("/tmp/dremel.pdf")

# blob_out = bucket.blob("images_upload/d.jpeg")
# blob_out.upload_from_string(bts)

In [3]:
# @bpd.remote_function([str], str, reuse=False, packages=["google-cloud-storage", "parse", "opencv-python"])
# def garrettwu_gcs(uri_in):
#     from google.cloud import storage
#     storage_client = storage.Client()    
#     bucket = storage_client.bucket("garrettwu_bucket")
#     blob_in = bucket.blob("images/download.jpeg")
#     bts = blob_in.download_as_bytes()

#     blob_out = bucket.blob(f"images_upload/remote_variable_func2.jpeg")
#     blob_out.upload_from_string(bts)

#     return "1"

In [10]:
dst_folder = "images_blur_upload_remote"

def parse_gcs_path(path):
    result = parse.parse("gs://{0}/{1}", path)

    return tuple(result)

@bpd.remote_function([str], str, packages=["numpy", "google-cloud-storage", "parse", "opencv-python"])
def garrettwu_test1(uri_in):
    import os
    import cv2 as cv
    import numpy as np
    from google.cloud import storage
    import parse
    storage_client = storage.Client()

    bucket_name, blob_path_in = parse_gcs_path(uri_in)
    bucket = storage_client.bucket(bucket_name)
    blob_in = bucket.blob(blob_path_in)
    bts = blob_in.download_as_bytes()

    nparr = np.frombuffer(bts, np.uint8)
    img = cv.imdecode(nparr, cv.IMREAD_UNCHANGED)
    img_blurred = cv.blur(img, (15, 15))
    bts = cv.imencode(".jpeg", img_blurred)[1].tobytes()

    file_name = uri_in[uri_in.rfind("/") + 1 :]

    blob_path_out = os.path.join(dst_folder, file_name)
    blob_out = bucket.blob(blob_path_out)
    blob_out.upload_from_string(bts)

    return f"gs://{bucket_name}/{blob_path_out}"

In [4]:
garrettwu_test.bigframes_cloud_function

'projects/bigframes-dev/locations/us-central1/functions/bigframes-7ed9f719809a94e2b41052de3becbfa9'

In [5]:
garrettwu_test.bigframes_remote_function

'bigframes-dev._76f0f906c2e04e83c3496619541347a5922c80ee.bigframes_7ed9f719809a94e2b41052de3becbfa9'

In [6]:
type(garrettwu_test)

function

In [8]:
df = bpd.DataFrame({"uri": ["gs://garrettwu_bucket/images/images.jpeg"]})

In [9]:
df["blurred"] = df["uri"].apply(garrettwu_test1)

BadRequest: 400 Received response code 400 from endpoint https://bigframes-bc2ee5cab9a065d6ec6b9302d1f3d64c-7krlje3eoq-uc.a.run.app with response {"errorMessage":"parse_gcs_path() takes 1 positional argument but 2 were given"}
.

Location: US
Job ID: acdee165-c4a0-436f-b7ae-41da68ee98af
 Share your usecase with the BigQuery DataFrames team at the https://bit.ly/bigframes-feedback survey.

In [7]:
df

,uri,blurred
0,gs://garrettwu_bucket/images/images.jpeg,gs://garrettwu_bucket/images_blur_upload_remot...


In [ ]:
out_file = "images_upload/remote_variable_alt.jpeg"


In [ ]:
df["a"].apply(garrettwu_gcs)

0    1
Name: a, dtype: string

In [ ]:
class A:

    def wrap_func(self):

        @bpd.remote_function([str], str, reuse=False, packages=["google-cloud-storage"])
        def _garrettwu_gcs(uri_in):
            from google.cloud import storage
            storage_client = storage.Client()    
            bucket = storage_client.bucket("garrettwu_bucket")
            blob_in = bucket.blob("images/download.jpeg")
            bts = blob_in.download_as_bytes()

            blob_out = bucket.blob(f"images_upload/remote_variable_func3.jpeg")
            blob_out.upload_from_string(bts)

            return "1"

UnsupportedTypeError: (<class '__main__.A'>, {<class 'bool'>, <class 'int'>, <class 'str'>, <class 'float'>})

In [ ]:
a = A()

In [ ]:
df["a"].apply(a.garrettwu_gcs)

SignatureValidationError: garrettwu_gcs(<__main__.A object at 0x7f8f598e5e70>, r0 := InMemoryTable
  data:
    PandasDataFrameProxy:
        level_0 column_0  bigframes_ordering_id
      0       0        0                      0

r1 := Selection[r0]
  selections:
    level_0:  r0.level_0
    column_0: r0.column_0

column_0: r1.column_0) too many positional arguments

Expected signature: garrettwu_gcs(uri_in: string)

In [ ]:
class B:
    def func_B(self):
        print("func B")
        def func_C():
            print("func C")
        func_C()

In [ ]:
B.func_B()

TypeError: B.func_B() missing 1 required positional argument: 'self'

In [ ]:
b = B()

In [ ]:
b.func_B()

func B
func C
